In [8]:
#Single Machine Scheduling

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
single_mach = pd.DataFrame({"Arrival Time" : [0,0,0,0,0,0,0,0,0,0],
                            'Job Number':[1,2,3,4,5,6,7,8,9,10], 
                            'Processing Time':[8,2,2,8,8,4,6,9,5,10], 
                            'Due Date':[4,13,2,8,5,17,6,19,7,8]})
single_mach

,Arrival Time,Job Number,Processing Time,Due Date
0,0,1,8,4
1,0,2,2,13
2,0,3,2,2
3,0,4,8,8
4,0,5,8,5
5,0,6,4,17
6,0,7,6,6
7,0,8,9,19
8,0,9,5,7
9,0,10,10,8


In [10]:
#Order according to EDD
single_mach = single_mach.sort_values('Due Date')
single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
single_mach["Lateness"] = single_mach["Completion Time"] - single_mach["Due Date"]


single_mach

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
2,0,3,2,2,2,0,0
0,0,1,8,4,10,6,6
4,0,5,8,5,18,13,13
6,0,7,6,6,24,18,18
8,0,9,5,7,29,22,22
3,0,4,8,8,37,29,29
9,0,10,10,8,47,39,39
1,0,2,2,13,49,36,36
5,0,6,4,17,53,36,36
7,0,8,9,19,62,43,43


In [11]:
import random
# create random df following this code: l = [random.randint(0,10) for i in range(5)]

random_jobs = pd.DataFrame({"Arrival Time":[40 + random.randint(10,40) for i in range(10)],
                            'Job Number' : [11,12,13,14,15,16,17,18,19,20],
                            'Processing Time':[random.randint(1,10) for i in range(10)], 
                            'Due Date':[7 + random.randint(0,25) for i in range(10)],
                             "Completion Time" : [0,0,0,0,0,0,0,0,0,0],
                             "Tardiness" : [0,0,0,0,0,0,0,0,0,0],
                             "Lateness" : [0,0,0,0,0,0,0,0,0,0]})
random_jobs = random_jobs.sort_values('Arrival Time')


random_jobs

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
0,51,11,1,9,0,0,0
5,53,16,4,8,0,0,0
8,54,19,9,12,0,0,0
1,58,12,8,22,0,0,0
2,61,13,10,27,0,0,0
7,67,18,5,20,0,0,0
9,69,20,3,18,0,0,0
4,70,15,4,29,0,0,0
3,77,14,10,29,0,0,0
6,80,17,2,29,0,0,0


In [12]:
#Optimizing for EDD (Earliest Due Date)

#Step 1. 

pending_jobs = pd.DataFrame()
reschedule = pd.DataFrame()

while random_jobs[random_jobs['Due Date'] > 0].shape[0] > 0:   

    random_jobs = random_jobs.sort_values('Arrival Time')

    print(random_jobs.iloc[0,0])

    pending_jobs = single_mach.loc[(single_mach["Completion Time"] > random_jobs.iloc[0,0])]
    pending_jobs = pending_jobs.reset_index()


    print(random_jobs.iloc[0])

    pending_jobs.drop(["index"], axis = 1, inplace=True)

    pending_jobs["Arrival Time"] = pending_jobs.iloc[0,4]
    pending_jobs = pending_jobs.drop([0] , axis = 0)
    single_mach = single_mach[~single_mach['Job Number'].isin(pending_jobs['Job Number'])]
    
    print("Pending_jobs")
    print(pending_jobs)
    
    #Paso los pending jobs a random jobs
    random_jobs = random_jobs.append(pending_jobs)
    pending_jobs = pending_jobs[0:0]
    
    

    #Step 4

    print(single_mach.iloc[-1,4])

    reschedule = random_jobs.loc[(random_jobs["Arrival Time"] <= single_mach.iloc[-1,4])]

    print("Reschedule")
    print(reschedule)


    #Step 5

    reschedule = reschedule.sort_values("Due Date")


    #Step 6

    random_jobs = random_jobs[~random_jobs['Job Number'].isin(reschedule['Job Number'])]

    random_jobs = random_jobs.sort_values("Arrival Time")
    
    print("Randomjobs al terminar Step 6")
    print(random_jobs)


    #Step 7

    single_mach = single_mach.append(reschedule)

    reschedule = reschedule[~reschedule['Job Number'].isin(single_mach['Job Number'])]

    print("Reschedule")
    print(reschedule)

    #Recompute Fi, Tardiness and Li

    single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
    single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
    single_mach["Lateness"] = single_mach["Completion Time"] - single_mach["Due Date"]

    print("Single_mach")
    print(single_mach)

51
Arrival Time       51
Job Number         11
Processing Time     1
Due Date            9
Completion Time     0
Tardiness           0
Lateness            0
Name: 0, dtype: int64
Pending_jobs
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
1            53           8                9        19               62   

   Tardiness  Lateness  
1         43        43  
53
Reschedule
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
0            51          11                1         9                0   
5            53          16                4         8                0   
1            53           8                9        19               62   

   Tardiness  Lateness  
0          0         0  
5          0         0  
1         43        43  
Randomjobs al terminar Step 6
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
8            54          19                9        12                0   
1           

Single_mach
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
2             0           3                2         2                2   
0             0           1                8         4               10   
4             0           5                8         5               18   
6             0           7                6         6               24   
8             0           9                5         7               29   
3             0           4                8         8               37   
9             0          10               10         8               47   
1             0           2                2        13               49   
5             0           6                4        17               53   
5            53          16                4         8               57   
1            57          11                1         9               58   
8            54          19                9        12               67   
1            

In [13]:
single_mach

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
2,0,3,2,2,2,0,0
0,0,1,8,4,10,6,6
4,0,5,8,5,18,13,13
6,0,7,6,6,24,18,18
8,0,9,5,7,29,22,22
3,0,4,8,8,37,29,29
9,0,10,10,8,47,39,39
1,0,2,2,13,49,36,36
5,0,6,4,17,53,36,36
5,53,16,4,8,57,49,49


In [15]:
#Performance Metrics with EDD

f_bar = single_mach["Completion Time"].mean()
t_bar = single_mach["Processing Time"].mean()
mean_waiting_time = f_bar-t_bar

t_max = single_mach["Tardiness"].max()

pos_count = [num for num in single_mach["Lateness"] if num >= 1] 
late_jobs = len(pos_count)

print("Mean Waiting time = "+ str(mean_waiting_time))
print("T Max = "+ str(t_max))
print("Number of Late Jobs = "+ str(late_jobs))

Mean Waiting time = 55.1
T Max = 89
Number of Late Jobs = 19
